In [1]:
from sqlalchemy import *

In [2]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
from pyspark.sql import functions as F, DataFrame
import datetime as dt
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
import pyspark.sql.types as t
from decimal import Decimal
from pyspark.sql.functions import regexp_replace
import os

In [3]:
from dataproc_sdk.dataproc_sdk_datiopysparksession.datiopysparksession import DatioPysparkSession
datioSparkSession = DatioPysparkSession().get_or_create()

from dataproc_sdk.dataproc_sdk_datiopysparksession import datiopysparksession
dataproc = datiopysparksession.DatioPysparkSession().get_or_create()

from dataproc_sdk.dataproc_sdk_schema.datioschema import DatioSchema
from dataproc_sdk.dataproc_sdk_datiofilesystem.datiofilesystem import DatioFileSystem

In [4]:
# para evitar problemas de tipología de datos
spark.conf.set("spark.sql.parquet.enableVectorizedReader", "false")

# Funciones

**hacer que la función no tome los csv para calcular la fecha, que se quede solo en el primer nivel de closing date**

In [5]:
# # calculamos la fecha más reciente de la ruta tomando como campo de particion el pasado como parámetro
# def last_partition (path:str, campo:str):
#     window = Window.partitionBy().orderBy(F.col(campo).desc())
    
#     # opcion 1
#     df = dataproc.read().parquet(path
#                                ).withColumn("rn", F.row_number().over(window)
#                                ).where(F.col("rn") == 1).drop('rn')
    
#     # opcion 2
#     # df = dataproc.read().parquet(path).coalesce(1
#     #                                            ).orderBy(F.col(campo).asc()
#     #                                            ).groupBy().agg(F.last(campo).alias(campo))
    
#     fecha = [x[campo] for x in df.select(campo).collect()][0]
#     print('Fecha de datos:',fecha)
#     return fecha
    

In [6]:
# calculamos la fecha más reciente de la ruta tomando como campo de particion el pasado como parámetro
def last_partition (p_path:str, campo:str):
    
    datio_path = DatioFileSystem().get().qualify(p_path)
    fs = datio_path.fileSystem()
    path = datio_path.path()
    path_list = fs.listStatus(path)
    paths = [path.getPath().toString() for path in path_list] #listado de todos los paths de la ruta pasada
    
    l_fechas = [element.split(campo+'=')[1] for element in paths if campo in element] #listado de todas las fechas
    return max(l_fechas) # fecha mayor

# Configuracion

In [7]:
# Ruta Raiz en SBX
root_path = '/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/'

In [8]:
mode_write= 'overwrite'# 'append'
# f_write = '2024-06-18' # particion de fechas

In [9]:
f_write = last_partition (root_path, 'closing_date')
root_pathc = root_path + 'closing_date=' + str(f_write)
root_pathc

'/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/closing_date=2024-10-15'

In [10]:
# Rutas datos en SBX
path_facilities = root_pathc + '/facilities'

path_limites_only = root_pathc + '/limites'

path_constantes = root_pathc + '/constants'

path_excluidas = root_pathc + '/facilities_excluded'

path_facilities_total = root_pathc + '/cartera_titulizar' # foto con toda la traza de las operaciones, para poder seleccionar las columnas necesarias

In [11]:
SCHEMA = 'joystick'

TABLES = {'joystick.securitization_limit':path_limites_only,
         'joystick.securitization_constant':path_constantes,
         'joystick.securitization_facilities_excluded':path_excluidas,
         'joystick.securitization_facilities_completed':path_facilities_total,
         # 'joystick.securitization_facilities:
         } 

In [12]:
# INDICES = {'i_piloto': ['BEC_CDM_RELACIONES_POSTG','BEC_CDM_RELACIONES_CAPT_POSTG',
#                         'BEC_CDM_PILOTOS_POSTG','BEC_CDM_PILOTOS_CAPT_POSTG'],
#           'i_relacionada': ['BEC_CDM_RELACIONES_POSTG','BEC_CDM_RELACIONES_CAPT_POSTG',
#                             'BEC_CDM_RELACIONADAS_POSTG','BEC_CDM_RELACIONADAS_CAPT_POSTG'],
#           'i_oficina': ['BEC_CDM_PILOTOS_POSTG','BEC_CDM_PILOTOS_CAPT_POSTG'],
#           'i_gestor': ['BEC_CDM_PILOTOS_POSTG','BEC_CDM_PILOTOS_CAPT_POSTG'],}

## Postgres

In [13]:
user = os.getenv("JPY_USER")
jpy = os.getenv("JPY_BASE_URL")
uuaa = jpy.split("/")[2]
vault = os.getenv("VAULT_HOSTS")
environ = vault.split(".")[3]
country = vault.split(".")[5]

In [14]:
print('user:',user)
print('jpy:',jpy)
print('uuaa:',uuaa)
print('vault:',vault)
print('environ:',environ)
print('country:',country)

user: E056745
jpy: /services/dslb/intelligence/user/E056745/
uuaa: dslb
vault: vault.service.eos.live01.daas.gl.igrupobbva
environ: live01
country: gl


In [15]:
# Define here the postgres host, port and database                        
#postgres_host = "ep-w-pgreporting.marathon.l4lb." + environ + ".daas." + country + ".igrupobbva:81"
postgres_host = "ep-w-pgreporting.sandbox.extservices.platform.marathon.mesos:5432"
postgres_db = uuaa
print(postgres_host + "/" + postgres_db)

# Path to needed certificates taken from Spark properties
sslKey = "/opt/sds/stratio_analytic/security/" + user + ".key"
sslCert = "/opt/sds/stratio_analytic/security/"  + user + ".pem"
sslRootCert = "/opt/sds/stratio_analytic/security/ca-bundle.pem"

pgurl = ("postgresql://{user}@{host}/{db}?sslmode=verify-full&sslcert={sslCert}&sslrootcert={sslRootCert}&sslkey={sslKey}"
               .format(sslCert=sslCert,
                       sslKey=sslKey,
                       sslRootCert=sslRootCert,
                       user=user,
                       host=postgres_host,
                       db=postgres_db))
pg = create_engine(pgurl)


sslCertdbc = spark.conf.get("spark.ssl.datastore.certPem.path")
sslKeydbc = spark.conf.get("spark.ssl.datastore.keyPKCS8.path")
sslRootCertdbc = spark.conf.get("spark.ssl.datastore.caPem.path")

pgjdbc = ("jdbc:postgresql://{host}/{db}?user={user}&ssl=true&sslmode=verify-full&sslcert={sslCert}&sslrootcert={sslRootCert}&sslkey={sslKey}&prepareThreshold=0"
               .format(sslCert=sslCertdbc,
                       sslKey=sslKeydbc,
                       sslRootCert=sslRootCertdbc,
                       user=user,
                       host=postgres_host,
                       db=postgres_db))

ep-w-pgreporting.sandbox.extservices.platform.marathon.mesos:5432/dslb


# Postgre

## Borramos Tablas

In [16]:
for k,v in TABLES.items():
    try:
        pg.execute("drop table "+k)
        print('Tabla borrada:',k)
    except:
        None

Tabla borrada: joystick.securitization_limit
Tabla borrada: joystick.securitization_constant
Tabla borrada: joystick.securitization_facilities_excluded
Tabla borrada: joystick.securitization_facilities_completed


In [17]:
# Generamos el esquema donde se van a almacenar las tablas
try:
    pg.execute('CREATE SCHEMA ' + SCHEMA)
    print('Esquema generado:',SCHEMA)
except:
    None

## Generamos Tablas

In [18]:
for k,v in TABLES.items():
    try:
        print('**** TABLA: ',k,'*****')
        print('**** path de datos: ',v,'*****')
        # leemos dataframe
        df = spark.read.parquet(v)
#         df.show(5,False)
        df.write.option("truncate","true").jdbc(url=pgjdbc,table=k,mode=mode_write)        
        print('numero de registros:',df.count())

    except:
        None

**** TABLA:  joystick.securitization_limit *****
**** path de datos:  /data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/closing_date=2024-10-15/limites *****
numero de registros: 216
**** TABLA:  joystick.securitization_constant *****
**** path de datos:  /data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/closing_date=2024-10-15/constants *****
numero de registros: 12
**** TABLA:  joystick.securitization_facilities_excluded *****
**** path de datos:  /data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/closing_date=2024-10-15/facilities_excluded *****
numero de registros: 171
**** TABLA:  joystick.securitization_facilities_completed *****
**** path de datos:  /data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/closing_date=2024-10-15/cartera_titulizar *****
numero de registros: 1023


## Añadir Indices

In [19]:
# for i in INDICES.keys():
#     pg.execute('DROP INDEX '+ i)
#     print('indice borrado ',i)

In [20]:
# for k,v in TABLES.items():
#     try:
#         print('tabla:',k)
# #         if(k in INDICES['i_piloto']):
# #             pg.execute('CREATE INDEX i_piloto ON ' + k + '(cif_piloto)')
# #             print('indice i_piloto generado en tabla ',k)
            
# #         if(k in INDICES['i_relacionada']):
# #             pg.execute('CREATE INDEX i_relacionada ON ' + k + '(cif_relacionada)')
# #             print('indice i_relacionada generado en tabla ',k)
            
#         if(k in INDICES['i_oficina']):
#             pg.execute('CREATE INDEX i_oficina ON ' + k + '(oficina_desc_piloto)')
#             print('indice i_oficina generado en tabla ',k)
            
#         if(k in INDICES['i_gestor']):
#             pg.execute('CREATE INDEX i_gestor ON ' + k + '(gestor_piloto)')
#             print('indice i_gestor generado en tabla ',k)
#     except Exception:
#         print('Se ha producido un error')
#         pass
# #     except:
# #         print('Se ha producido un error')
# #         continue

In [21]:
# sorted(spark.read.parquet(path_pilotos_postg_cap).columns)

# TEST

In [22]:
# query = "SELECT * FROM joystick.concentration_risk_commitment_P WHERE group_id='G00000000001233'"
query = "SELECT * FROM joystick.securitization_limit"

## Primera fila

In [23]:
pt = pg.execute(query)
pt.first()

('escenario model verano IV', datetime.date(2024, 6, 24), 'rating_sp', 'rating_group', 'CC+', None, None, 0.0, 1, 1, 'individual', 1, 0, 0, 40, '2024-10-15')

## Todas las filas

In [24]:
def rows_as_dicts(cursor):
    """convert tuple result to dict with cursor"""
    col_names = [i[0] for i in cursor.description]
    return [dict(zip(col_names, row)) for row in cursor]

cursor = pg.execute(query).cursor
result = rows_as_dicts(cursor)

for k in result:
    print(k)
# result

{'name_list_desc': 'escenario model verano IV', 'limit_date': datetime.date(2024, 6, 24), 'limit_type': 'rating_sp', 'concept1_desc': 'rating_group', 'concept1_value': 'CC+', 'concept2_desc': None, 'concept2_value': None, 'limit_value': 0.0, 'corporate_loan_flag': 1, 'project_finance_flag': 1, 'limit_scope': 'individual', 'active_flag': 1, 'visual_order': 0, 'complex_limit': 0, 'id_limit': 40, 'closing_date': '2024-10-15'}
{'name_list_desc': 'escenario model verano IV', 'limit_date': datetime.date(2024, 6, 24), 'limit_type': 'rating_sp', 'concept1_desc': 'rating_group', 'concept1_value': 'CCC', 'concept2_desc': None, 'concept2_value': None, 'limit_value': 0.0, 'corporate_loan_flag': 1, 'project_finance_flag': 1, 'limit_scope': 'individual', 'active_flag': 1, 'visual_order': 0, 'complex_limit': 0, 'id_limit': 38, 'closing_date': '2024-10-15'}
{'name_list_desc': 'escenario model verano IV', 'limit_date': datetime.date(2024, 6, 24), 'limit_type': 'maturity_min', 'concept1_desc': 'num_days

## Tablas origen

In [25]:
df = spark.read.parquet('/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/closing_date='+f_write+'/limites')

In [26]:
df.show(5,False)

+-------------------------+----------+------------------+--------------+---------------------------------------+-------------+--------------+-----------+-------------------+--------------------+-----------+-----------+------------+-------------+--------+------------+
|name_list_desc           |limit_date|limit_type        |concept1_desc |concept1_value                         |concept2_desc|concept2_value|limit_value|corporate_loan_flag|project_finance_flag|limit_scope|active_flag|visual_order|complex_limit|id_limit|closing_date|
+-------------------------+----------+------------------+--------------+---------------------------------------+-------------+--------------+-----------+-------------------+--------------------+-----------+-----------+------------+-------------+--------+------------+
|escenario model verano IV|2024-06-24|customer_subsector|subsector_desc|Paper, plastic, metal & glass packaging|null         |null          |0.5        |1                  |0                   |po